In [1]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format ,from_unixtime ,monotonically_increasing_id
from pyspark.sql import types as t
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1568404209694_0023,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .enableHiveSupport().getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# read  PORT statstics data
df = spark.read.format("parquet").load("s3n://capstone-rk/demography-by-state")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- state: string (nullable = true)
 |-- code: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Males: string (nullable = true)
 |-- females: string (nullable = true)
 |-- tot_Pop: string (nullable = true)
 |-- Veterans: string (nullable = true)
 |-- Foreign: string (nullable = true)
 |-- Household: string (nullable = true)
 |-- Hispanic: string (nullable = true)
 |-- Asian: string (nullable = true)
 |-- White: string (nullable = true)
 |-- Black: string (nullable = true)
 |-- Native: string (nullable = true)

In [30]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------+----+-----+
|            state|visitors|year|month|
+-----------------+--------+----+-----+
|DIST. OF COLUMBIA|   27859|2016|    6|
|DIST. OF COLUMBIA|   38223|2016|    7|
|   VIRGIN ISLANDS|     343|2016|    6|
|   VIRGIN ISLANDS|     410|2016|    7|
|    MASSACHUSETTS|  105193|2016|    6|
|    NEW HAMPSHIRE|    6366|2016|    6|
|    MASSACHUSETTS|  131524|2016|    7|
|    NEW HAMPSHIRE|    7663|2016|    7|
|     PENNSYLVANIA|   43294|2016|    6|
|     RHODE ISLAND|    6293|2016|    6|
|     PENNSYLVANIA|   50897|2016|    7|
|     RHODE ISLAND|    6434|2016|    7|
|      CONNECTICUT|   21275|2016|    6|
|      MISSISSIPPI|    2356|2016|    6|
|      N. CAROLINA|   26373|2016|    6|
|      PUERTO RICO|    9182|2016|    6|
|      S. CAROLINA|    9783|2016|    6|
|      W. VIRGINIA|    1374|2016|    6|
|      CONNECTICUT|   24193|2016|    7|
|      MISSISSIPPI|    2568|2016|    7|
+-----------------+--------+----+-----+
only showing top 20 rows

In [31]:
# PIVOT the DATA by month
# We can do this by Year too. Since we have loaded data for only one year, we are pivoting by just month
df2 = df.groupby("state","Year").pivot("month").avg('visitors')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----+---------+--------+--------+
|        state|Year|     null|       6|       7|
+-------------+----+---------+--------+--------+
|    S. DAKOTA|2016|     null|  1322.0|  1583.0|
|     NEBRASKA|2016|     null| 26149.0| 27884.0|
|     COLORADO|2016|     null| 28878.0| 36357.0|
|  PUERTO RICO|2016|     null|  9182.0| 12132.0|
|         GUAM|2016|     null|105693.0| 60816.0|
|MASSACHUSETTS|2016|     null|105193.0|131524.0|
|      MONTANA|2016|     null|  4019.0|  6848.0|
|     WYOMING |2016|     null|  2236.0|  3195.0|
|         null|null|3697736.5|    null|    null|
|  CONNECTICUT|2016|     null| 21275.0| 24193.0|
|     MICHIGAN|2016|     null| 40897.0| 47101.0|
|    N. DAKOTA|2016|     null|  1340.0|  1859.0|
|     DELAWARE|2016|     null|  4147.0|  5541.0|
|      ARIZONA|2016|     null| 21620.0| 22146.0|
| RHODE ISLAND|2016|     null|  6293.0|  6434.0|
|  MISSISSIPPI|2016|     null|  2356.0|  2568.0|
|  W. VIRGINIA|2016|     null|  1374.0|  1694.0|
|        IDAHO|2016|

In [40]:

# add alias to column name
df3=df2.filter(df2["year"] != 0).select(df2.state.alias("Visiting-State"), df2.Year.alias("Year") , df2["6"].alias("June") , df2["7"].alias("July")  )
df5 =df3.fillna(0)
df5.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----+--------+--------+
|Visiting-State|Year|    June|    July|
+--------------+----+--------+--------+
|     S. DAKOTA|2016|  1322.0|  1583.0|
|      NEBRASKA|2016| 26149.0| 27884.0|
|      COLORADO|2016| 28878.0| 36357.0|
|   PUERTO RICO|2016|  9182.0| 12132.0|
|          GUAM|2016|105693.0| 60816.0|
| MASSACHUSETTS|2016|105193.0|131524.0|
|       MONTANA|2016|  4019.0|  6848.0|
|      WYOMING |2016|  2236.0|  3195.0|
|   CONNECTICUT|2016| 21275.0| 24193.0|
|      MICHIGAN|2016| 40897.0| 47101.0|
+--------------+----+--------+--------+
only showing top 10 rows

In [41]:
# compute % change
df4 = df5.withColumn("% change in July" , format_number( ((( df5["July"] - df5["June"]  ) /  df5["June"] ) * 100),2))
#df4 = df3.withColumn("% change in July" ,   ((( df3["July"] - df3["June"] )  /  df3["June"] ) * 100))
          

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df4.select ("Visiting-State", "year",format_number("June",0).alias("June") ,format_number("July",0).alias("July") , "% change in July"  ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----+-------+-------+----------------+
|   Visiting-State|year|   June|   July|% change in July|
+-----------------+----+-------+-------+----------------+
|        S. DAKOTA|2016|  1,322|  1,583|           19.74|
|         NEBRASKA|2016| 26,149| 27,884|            6.64|
|         COLORADO|2016| 28,878| 36,357|           25.90|
|      PUERTO RICO|2016|  9,182| 12,132|           32.13|
|             GUAM|2016|105,693| 60,816|          -42.46|
|    MASSACHUSETTS|2016|105,193|131,524|           25.03|
|          MONTANA|2016|  4,019|  6,848|           70.39|
|         WYOMING |2016|  2,236|  3,195|           42.89|
|      CONNECTICUT|2016| 21,275| 24,193|           13.72|
|         MICHIGAN|2016| 40,897| 47,101|           15.17|
|        N. DAKOTA|2016|  1,340|  1,859|           38.73|
|         DELAWARE|2016|  4,147|  5,541|           33.61|
|          ARIZONA|2016| 21,620| 22,146|            2.43|
|     RHODE ISLAND|2016|  6,293|  6,434|            2.24|
|      MISSISS